In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import os
import pickle 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import tensorflow as tf

In [3]:
data = pd.read_csv('data/hcsdatacleaned.csv').iloc[:,1:]
data

,From Date_3,To Date_3,PM2.5_3,RH_3,AT_3,WS_3,WD_3,lat_3,long_3,From Date_4,...,long_17,From Date_18,To Date_18,PM2.5_18,RH_18,AT_18,WS_18,WD_18,lat_18,long_18
0,26-07-2023 09:00,26-07-2023 09:15,12.0,76.0,31.2,3.0,229.0,25.262,87.011,26-07-2023 09:00,...,85.245,26-07-2023 09:00,26-07-2023 09:15,39.0,64.0,34.4,1.5,19.0,26.227,84.357
1,26-07-2023 10:00,26-07-2023 10:15,7.0,73.0,31.9,3.0,356.0,25.262,87.011,26-07-2023 10:00,...,85.245,26-07-2023 10:00,26-07-2023 10:15,25.0,62.0,35.1,1.6,29.0,26.227,84.357
2,26-07-2023 11:00,26-07-2023 11:15,2.0,69.0,32.7,3.5,353.0,25.262,87.011,26-07-2023 11:00,...,85.245,26-07-2023 11:00,26-07-2023 11:15,28.0,57.0,36.1,1.7,12.0,26.227,84.357
3,26-07-2023 12:00,26-07-2023 12:15,6.0,64.0,34.1,3.4,354.0,25.262,87.011,26-07-2023 12:00,...,85.245,26-07-2023 12:00,26-07-2023 12:15,25.0,56.0,36.9,1.5,31.0,26.227,84.357
4,26-07-2023 13:00,26-07-2023 13:15,15.0,62.0,34.7,3.3,358.0,25.262,87.011,26-07-2023 13:00,...,85.245,26-07-2023 13:00,26-07-2023 13:15,23.0,57.0,37.2,1.5,354.0,26.227,84.357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,16-08-2023 10:00,16-08-2023 10:15,36.0,76.0,32.8,4.3,353.0,25.262,87.011,16-08-2023 10:00,...,85.245,16-08-2023 10:00,16-08-2023 10:15,40.0,60.0,35.6,1.8,227.0,26.227,84.357
506,16-08-2023 11:00,16-08-2023 11:15,30.0,74.0,33.6,4.3,353.0,25.262,87.011,16-08-2023 11:00,...,85.245,16-08-2023 11:00,16-08-2023 11:15,40.0,60.0,35.6,1.8,227.0,26.227,84.357
507,16-08-2023 12:00,16-08-2023 12:15,34.0,70.0,33.4,4.2,355.0,25.262,87.011,16-08-2023 12:00,...,85.245,16-08-2023 12:00,16-08-2023 12:15,40.0,60.0,35.6,1.8,227.0,26.227,84.357
508,16-08-2023 13:00,16-08-2023 13:15,34.0,68.0,34.3,4.0,355.0,25.262,87.011,16-08-2023 13:00,...,85.245,16-08-2023 13:00,16-08-2023 13:15,40.0,60.0,35.6,1.8,227.0,26.227,84.357


In [29]:
lcs_data = pd.read_csv('data/AQI_LCS_data_prep.csv').iloc[:,1:]
lcs_data

,Time,PM25_1201230029_ARARIA_ARARIA,Temp_1201230029_ARARIA_ARARIA,RH_1201230029_ARARIA_ARARIA,lat_PM25_1201230029_ARARIA_ARARIA,long_PM25_1201230029_ARARIA_ARARIA,PM25_1201230033_KURSAKANTA_ARARIA,Temp_1201230033_KURSAKANTA_ARARIA,RH_1201230033_KURSAKANTA_ARARIA,lat_PM25_1201230033_KURSAKANTA_ARARIA,...,PM25_BCDDC247BFE3_BHITAHA_WCHAMPARAN,Temp_BCDDC247BFE3_BHITAHA_WCHAMPARAN,RH_BCDDC247BFE3_BHITAHA_WCHAMPARAN,lat_PM25_BCDDC247BFE3_BHITAHA_WCHAMPARAN,long_PM25_BCDDC247BFE3_BHITAHA_WCHAMPARAN,PM25_DC4F22364445_PIPRASI_WCHAMPARAN,Temp_DC4F22364445_PIPRASI_WCHAMPARAN,RH_DC4F22364445_PIPRASI_WCHAMPARAN,lat_PM25_DC4F22364445_PIPRASI_WCHAMPARAN,long_PM25_DC4F22364445_PIPRASI_WCHAMPARAN
0,2023-07-26 09:00:00,8.0,32.0,98.0,26.148,87.457,26.0,30.5,90.0,26.359,...,14.55,38.25,50.36,26.907,84.136,18.02,35.59,57.30,27.06,84.023
1,2023-07-26 10:00:00,6.0,33.0,98.0,26.148,87.457,20.5,30.5,90.0,26.359,...,11.22,38.82,50.36,26.907,84.136,13.92,36.46,57.30,27.06,84.023
2,2023-07-26 11:00:00,6.5,34.5,98.0,26.148,87.457,12.5,30.0,90.0,26.359,...,15.26,38.46,50.36,26.907,84.136,14.37,35.90,57.30,27.06,84.023
3,2023-07-26 12:00:00,13.0,33.5,98.0,26.148,87.457,15.5,30.0,88.0,26.359,...,37.81,39.47,47.59,26.907,84.136,11.13,36.68,56.93,27.06,84.023
4,2023-07-26 13:00:00,10.0,34.0,98.0,26.148,87.457,10.0,32.0,82.5,26.359,...,15.55,39.96,46.83,26.907,84.136,11.48,38.89,51.65,27.06,84.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2023-08-16 10:00:00,41.0,34.5,98.0,26.148,87.457,30.5,41.5,56.5,26.359,...,30.40,36.23,70.98,26.907,84.136,33.65,35.33,65.03,27.06,84.023
506,2023-08-16 11:00:00,36.0,35.5,98.0,26.148,87.457,30.0,41.5,54.0,26.359,...,28.02,38.29,62.14,26.907,84.136,33.32,35.55,64.64,27.06,84.023
507,2023-08-16 12:00:00,29.0,36.0,98.0,26.148,87.457,30.5,41.5,54.5,26.359,...,28.00,38.60,62.60,26.907,84.136,30.00,36.20,60.80,27.06,84.023
508,2023-08-16 13:00:00,21.0,36.0,98.0,26.148,87.457,29.0,41.0,57.0,26.359,...,28.00,38.60,62.60,26.907,84.136,30.00,36.20,60.80,27.06,84.023


In [9]:
data.shape[1]/9

11.0

In [11]:
(lcs_data.shape[1]-1)/5

247.0

In [21]:
lcs_sensors = {}
for i in range(247):
    sensor = lcs_data.columns[5*i].split('_')[1]
    loc = (lcs_data.iloc[0,5*i+3],lcs_data.iloc[0,5*i+4])
    lcs_sensors[sensor]=loc
    

In [23]:
hcs_sensors = {}
for i in range(11):
    sensor = data.columns[9*i+2].split('_')[1]
    loc = (data.iloc[0,9*i+7],data.iloc[0,9*i+8])
    hcs_sensors[sensor] = loc

In [27]:
matching = {}
for sensor in lcs_sensors.keys():
    for tgt in hcs_sensors.keys():
        if lcs_sensors[sensor]== hcs_sensors[tgt]:
            matching[sensor] = tgt

In [28]:
matching

{'1212220152': '3',
 '1201230089': '8',
 '1203230019': '9',
 '1211220066': '18',
 '4C75250A4545': '17'}

In [31]:
coll_data = pd.DataFrame()
columns = ['Time']
for col in lcs_data.columns:
    if(col=='Time'):
        continue
    if col.split('_')[1] in matching.keys():
        columns.append(col)

In [35]:
columns

['Time',
 'PM25_1212220152_JAGDISHPUR.BG_BHAGALPUR',
 'Temp_1212220152_JAGDISHPUR.BG_BHAGALPUR',
 'RH_1212220152_JAGDISHPUR.BG_BHAGALPUR',
 'PM25_1201230089_KATIHAR_KATIHAR',
 'Temp_1201230089_KATIHAR_KATIHAR',
 'RH_1201230089_KATIHAR_KATIHAR',
 'PM25_1203230019_KISHANGANJ_KISHANGANJ',
 'Temp_1203230019_KISHANGANJ_KISHANGANJ',
 'RH_1203230019_KISHANGANJ_KISHANGANJ',
 'PM25_1211220066_SIWAN_SIWAN',
 'Temp_1211220066_SIWAN_SIWAN',
 'RH_1211220066_SIWAN_SIWAN',
 'PM25_4C75250A4545_HAJIPUR_VAISHALI',
 'Temp_4C75250A4545_HAJIPUR_VAISHALI',
 'RH_4C75250A4545_HAJIPUR_VAISHALI']

In [36]:
locations = ['BHAGALPUR','KATIHAR','SIWAN','KISHANGANJ','HAJIPUR']
data_dict = {}
for loc in locations:
    col_to_append = [col for col in columns if loc in col]
    df = pd.DataFrame()
    df['Time'] = lcs_data['Time']
    for c in col_to_append:
        if 'PM25' in c:
            df['PM25L'] = lcs_data[c]
        if 'Temp' in c:
            df['Temp'] = lcs_data[c]
        if 'RH' in c:
            df['RH'] = lcs_data[c]
    sensor = col_to_append[0].split('_')[1]
    matched = matching[sensor]
    df['PM25H'] = data[f'PM2.5_{matched}']

    data_dict[loc] = df

In [45]:
with open('data/collocated_sensor_dict.pkl','wb') as f:
    pickle.dump(data_dict,f)

In [2]:
with open('data/collocated_sensor_dict.pkl','rb') as f:
    my_dict = pickle.load(f)

In [3]:
locations = [['BHAGALPUR','KATIHAR','SIWAN','HAJIPUR']]

In [53]:
my_dict['BHAGALPUR']

,Time,PM25L,Temp,RH,PM25H
0,2023-07-26 09:00:00,10.0,34.0,63.5,12.0
1,2023-07-26 10:00:00,8.5,35.5,63.5,7.0
2,2023-07-26 11:00:00,7.0,37.0,63.5,2.0
3,2023-07-26 12:00:00,9.0,38.0,60.0,6.0
4,2023-07-26 13:00:00,8.0,40.0,54.5,15.0
...,...,...,...,...,...
505,2023-08-16 10:00:00,39.5,35.5,77.0,36.0
506,2023-08-16 11:00:00,30.5,37.5,72.5,30.0
507,2023-08-16 12:00:00,32.5,38.5,69.0,34.0
508,2023-08-16 13:00:00,28.5,39.5,64.5,34.0


In [4]:
scores = {}

In [45]:
test_loc = 'SIWAN'

In [46]:
df_test = my_dict[test_loc].iloc[:,1:]
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_test),columns=df_test.columns)
X_test = df_scaled.iloc[:,:-1].values
y_test = df_scaled.iloc[:,-1].values

In [47]:
df = pd.concat([my_dict['BHAGALPUR'],my_dict['HAJIPUR'],my_dict['KATIHAR']]).iloc[:,1:]


In [48]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df),columns = df.columns)
X = df_scaled.iloc[:,:-1].values
y = df['PM25H'].values

In [49]:
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8)
print(f"X_train.shape {X_train.shape}")
print(f"y_train.shape {y_train.shape}")
print(f"X_val.shape {X_val.shape}")
print(f"y_val.shape {y_val.shape}")
print(f"X_test.shape {X_test.shape}")
print(f"y_test.shape {y_test.shape}")

X_train.shape (1224, 3)
y_train.shape (1224,)
X_val.shape (306, 3)
y_val.shape (306,)
X_test.shape (510, 3)
y_test.shape (510,)


In [52]:
import datetime
log_dir = "logs/nn_baselines/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(3,), activation=None),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.8),  # Example dropout rate of 0.5
    
    tf.keras.layers.Dense(64, activation=None),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),  # Adjust dropout rate as needed

    tf.keras.layers.Dense(1)  
])
model.compile(
    optimizer='adam',
    loss='mean_squared_error',  
    metrics=['mean_absolute_error']
)

In [54]:
model.fit(X_train,y_train,epochs = 38,validation_data=(X_val,y_val),callbacks=tensorboard_callback)

Epoch 1/38
39/39 [==============================] - 1s 6ms/step - loss: 903.9349 - mean_absolute_error: 23.4106 - val_loss: 743.3211 - val_mean_absolute_error: 20.3916
Epoch 2/38
39/39 [==============================] - 0s 3ms/step - loss: 766.2484 - mean_absolute_error: 20.5621 - val_loss: 615.5136 - val_mean_absolute_error: 17.6804
Epoch 3/38
39/39 [==============================] - 0s 3ms/step - loss: 661.5845 - mean_absolute_error: 18.2588 - val_loss: 522.3427 - val_mean_absolute_error: 15.5575
Epoch 4/38
39/39 [==============================] - 0s 3ms/step - loss: 578.1948 - mean_absolute_error: 16.4719 - val_loss: 451.4385 - val_mean_absolute_error: 13.8606
Epoch 5/38
39/39 [==============================] - 0s 3ms/step - loss: 499.9856 - mean_absolute_error: 14.8153 - val_loss: 396.8010 - val_mean_absolute_error: 12.6499
Epoch 6/38
39/39 [==============================] - 0s 3ms/step - loss: 439.8465 - mean_absolute_error: 13.7214 - val_loss: 357.8662 - val_mean_absolute_error: 

In [55]:
from sklearn.metrics import r2_score, mean_absolute_error
y_pred = model.predict(X_test)
r2 = r2_score(y_pred,y_test)
print(f"r2-score {r2}")
mae = mean_absolute_error(y_pred,y_test)
print(f"mae score {mae}")

scores[test_loc] = {
    'r2':r2,
    'mae':mae

}

16/16 [==============================] - 0s 727us/step
r2-score -5.964431636649907
mae score 24.75619141447778


In [56]:
with open('logs/baselines/nn.json','w') as f:
    json.dump(scores,f)